# [Aligning Big Brains and Atlases](https://biop.github.io/ijp-imagetoatlas/) in Python

This series of notebook demoes the use of ABBA with python.

If you managed to create an environment with PyImageJ and DeepSlice, you will be able, by running this notebook, to perform a fully automated registration of demo mouse brain sections to the Adult Mouse Allen Brain atlas.

For this notebook to run, an atlas will need to be downloaded, as well as some sample dataset.

The dataset is downloaded from https://zenodo.org/record/6592478

Multichannel registration works reasonably well in this notebook because the first channel of the atlas (NISSL, indexed 0), is ressembling the DAPI channel of the slices (indexed 0), and the second channel of the atlas (ARA, indexed 1), is ressembling the autofluorescence channel of the slices (indexed 1).

NOTE : You need to set the elastix and transformix path for the elastix registration steps.

In [1]:
# core dependencies
import os
import time
from pathlib import Path

from abba import Abba

In [2]:
# Demo dataset for automated slices registration
zenodo_demo_slices_url = 'https://zenodo.org/record/6592478/files/'


# Only one section every five section is used for this demo
demo_sections = [
    'S00.tif',
    'S05.tif',
    'S10.tif',
    'S15.tif',
    'S20.tif',
    'S25.tif',
    'S30.tif',
    'S35.tif',
    'S40.tif',
    'S45.tif',
    'S50.tif',
    'S55.tif',
    'S60.tif',
    'S65.tif',
    'S70.tif',
    'S75.tif',
    'S80.tif']


def download_if_necessary(base_path, section_name):
    output_path = Path(base_path + section_name)
    if not output_path.exists():
        utils.check_internet_connection()
        url = zenodo_demo_slices_url + section_name + '?download=1'
        utils.retrieve_over_http(url, output_path)


def download_test_images(base_path):
    [download_if_necessary(base_path, section) for section in demo_sections]



## 1. Download test sections if necessary

    

In [3]:
basePath = os.getcwd() + '/images/'

if not os.path.exists(basePath):
    os.makedirs(basePath)
    
download_test_images(basePath)

## 2. ABBA initialization

In [4]:
headless = True



# -- FOR DEBUGGING
# import imagej.doctor
# imagej.doctor.checkup()
# imagej.doctor.debug_to_stderr()

import logging
logging.basicConfig(level=logging.DEBUG)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

if headless:
    # -- HEADLESS
    abba = Abba('Adult Mouse Brain - Allen Brain Atlas V3', headless=True, enable_jupyter_ui=True)
else:
    # -- NOT HEADLESS
    abba = Abba('Adult Mouse Brain - Allen Brain Atlas V3')
    abba.show_bdv_ui()  # creates and show a bdv view


DEBUG:imagej:List of Maven coordinates given: ['net.imagej:imagej:2.9.0', 'net.imagej:imagej-legacy:0.39.2', 'ch.epfl.biop:ImageToAtlasRegister:0.3.7']
DEBUG:scyjava:Adding jars from endpoints ['net.imagej:imagej:2.9.0+net.imagej:imagej-legacy:0.39.2+ch.epfl.biop:ImageToAtlasRegister:0.3.7', 'io.scif:scifio-labeling:0.3.1', 'net.imglib2:imglib2-imglyb:1.0.1']
DEBUG:scyjava:Using endpoints ['net.imagej:imagej:2.9.0+net.imagej:imagej-legacy:0.39.2+ch.epfl.biop:ImageToAtlasRegister:0.3.7', 'io.scif:scifio-labeling:0.3.1', 'net.imglib2:imglib2-imglyb:1.0.1']
DEBUG:scyjava.config:Getting verbose level: 0
DEBUG:jgo:Returning expanded coordinate net.imagej:imagej:2.9.0.
DEBUG:jgo:Returning expanded coordinate ch.epfl.biop:ImageToAtlasRegister:0.3.7.
DEBUG:jgo:Returning expanded coordinate io.scif:scifio-labeling:0.3.1.
DEBUG:jgo:Returning expanded coordinate net.imagej:imagej-legacy:0.39.2.
DEBUG:jgo:Returning expanded coordinate net.imglib2:imglib2-imglyb:1.0.1.
DEBUG:scyjava:Starting JVM
DE

Scijava jupyter ui enabled


DEBUG:scyjava:The JVM is already running.
DEBUG:ScijavaJupyterUI:Jupyter pre-processing, module ch.epfl.biop.atlas.mouse.allen.ccfv3.command.AllenBrainAdultMouseAtlasCCF2017Command
DEBUG:ScijavaJupyterUI:Unresolved input: mapUrl
DEBUG:ScijavaJupyterUI:Widget acquired for input mapUrl
DEBUG:ScijavaJupyterUI:Unresolved input: ontologyUrl
DEBUG:ScijavaJupyterUI:Widget acquired for input ontologyUrl
DEBUG:ScijavaJupyterUI:Display widgets module ch.epfl.biop.atlas.mouse.allen.ccfv3.command.AllenBrainAdultMouseAtlasCCF2017Command


DEBUG:ScijavaJupyterUI:Waiting for click
DEBUG:ScijavaJupyterUI:sleeping


java.util.concurrent.ExecutionException: java.util.concurrent.ExecutionException: java.lang.RuntimeException: Module threw exception

## 3. Import sections into ABBA

In [ ]:
# import sections into ABBA
files = [basePath + section for section in demo_sections]
abba.import_from_files(filepaths=files)

# ALL REGISTRATIONS AND COMMANDS BELOW ARE PERFORMED ON THE SELECTED SLICES!!
# since we want to register all of them, we select all of them
abba.select_all_slices()

In [ ]:
# we want to avoid saturation in the display. This does not matter for
# all registration methods EXCEPT for DeepSlice, which takes in rgb images
abba.change_display_settings(0, 0, 500)
abba.change_display_settings(1, 0, 1200)

## 4. DeepSlice Registration(s)

In [ ]:
# a first deepslice registration round : possible because it's the Allen CCF atlas, cut in coronal mode
# what's assumed : the sections are already in the correct order
abba.register_slices_deepslice(channels=[0, 1])

# second deepslice registration: because the slices are resampled for the registration,
# we usually get a slightly better positioning along z and cutting angle
# also: it's fast, and the combination of two affine transforms is
# an affine transform, so it's not like we are adding extra degrees of freedom
abba.register_slices_deepslice(channels=[0, 1])

## 5. Elastix affine registration

In [ ]:
# a round of elastix registration, affine
# the channel 0 of the dataset (DAPI) is registered with the Nissl Channel of the atlas (0)
# and the channel 1 of the dataset (mainly autofluo) is registered with the autofluo channel of the atlas (1)
# these two channels have equal weights in the registration process
abba.register_slices_elastix_affine(channels_slice_csv='0,1',
                                    channels_atlas_csv='0,1',
                                    pixel_size_micrometer=40)

## 6. Elastix spline registration

In [ ]:
# optional: a round of elastix registration, spline
# same channels as in the affine registration
# 5 control points along x = very coarse spline (and thus maybe unnecessary)
# abba.register_elastix_spline(
#    nb_control_points=5,
#    atlas_image_channels=[0, 1],
#    slice_image_channels=[0, 1],
#    pixel_size_micrometer=40).get()

# a round of elastix registration, affine
# same channels as in the affine registration 
# 16 control points = reasonable spline, which allows for local corrections, without deforming two much the section
abba.register_slices_elastix_spline(channels_slice_csv='0,1',
                                    channels_atlas_csv='0,1',
                                    nb_control_points_x=16,
                                    pixel_size_micrometer=20)

## 7. Wait for end of all registrations

In [ ]:
# all tasks/registrations are enqueued and executed asynchronously
# if you need to wait before saving, then wait for all tasks to be finished:
abba.wait_for_end_of_tasks()

## 8. Saving the result

In [ ]:
save_dir = os.path.join(os.getcwd(), 'temp', 'notebook0', 'state')

if not os.path.exists(save_dir):
    os.makedirs(save_dir)


abba.state_save(save_dir+"/state.json") # full absolute path needed

In [ ]:
import asyncio
import time

import ipywidgets as w
from IPython.display import display
from jupyter_ui_poll import run_ui_poll_loop, ui_events, with_ui_events

In [ ]:
def on_click(btn):
    n = int(btn.description)
    btn.description = str(n + 1)


def test_button():
    """
    Create button that displays number of times it was clicked
    """
    btn = w.Button(description="0")
    btn.on_click(on_click)
    return btn


display(test_button())

In [ ]:
btn = test_button()
print("Press this button 10 times to terminate")
display(btn)

with ui_events() as ui_poll:
    while int(btn.description) < 10:
        print(btn.description, end="")
        ui_poll(11)  # Process upto 11 ui events per iteration
        time.sleep(0.1)

print("... done")